# Welome to Tech Day NLP Demonstration

In [1]:
import pandas as pd
import numpy as np
import time
import os
from os import path
from string import punctuation
import re


import nltk

nltk.download

from nltk.corpus import stopwords

pd.options.display.max_columns = None

In [9]:
def read_data():
    df = pd.read_csv('./data/file.csv')
    
    # Create a temporary list of sources starting from the first instance of sources 
    # and returning unique values in the 1st column (the column containing source names)
    # This also pulls in extra instances of 'Source' in the dataframe
    temp_sources = list(df.iloc[((df[df[df.columns[1]] == 'Source'].index[0])+1):,1].unique())
    
    # Create source list and use a for loop to fill the source list
    # using the temporary source list but excluding 'Source'
    source_list = []
    for i in temp_sources:
        if i != 'Source':
            source_list.append(i.lower())
    
    # Sanity Check
    # print(source_list)
    
    # Determine the index values in the data frame where 'Source' 
    # is found in column 1. This is used later to slice the dataframe into
    # sub dataframes
    source_breaks = list(df[df[df.columns[1]] == 'Source'].index)
    
    # sanity check
    # print(source_breaks)
    
    # print(len(source_list), len(source_breaks))
    
    # create some empty holder lists and dictionaries; 'd' is going to be a dictionary of dataframes
    # The other temp lists and dictionary are used in creating these sub dataframes and associated column names
    d = {}
    bad_cols = []
    good_cols = []
    temp_dict = {}
    
    # Create a for loop to iterate through source_list to construct sub dataframes
    for i, vals in enumerate(source_list):
        
        # Create a temp var 'j' for indexing/slicing the dataframe with iloc
        j = i+1
        
        # Create an if/else to test if the value of 'j' exceeds the length of the source_list
        # If it does not, create some sub dataframes using the value of 'j' as the 'stop' point 
        # for iloc slice.
        #
        # If it does, use the length of the main dataframe as the 'stop' point for the iloc slice
        if j < len(source_list):
            d['{0}'.format(vals)] = df.iloc[source_breaks[i]+1: source_breaks[j]]
            
            # Rename the columns by creating a dictionary from the bad column names and
            # the actual column names found at a specific location in the main dataframe
            bad_cols = list(d['{0}'.format(vals)].columns)
            good_cols = list(df.iloc[source_breaks[i]])
            temp_dict = dict(zip(bad_cols, good_cols))
            
            d['{0}'.format(vals)].rename(columns=temp_dict, inplace=True)
            d['{0}'.format(vals)].reset_index(inplace=True, drop=True)
            
        else:
            d['{0}'.format(vals)] = df.iloc[source_breaks[i]+1:]
            
            # Rename the columns by creating a dictionary from the bad column names and
            # the actual column names found at a specific location in the main dataframe
            bad_cols = list(d['{0}'.format(vals)].columns)
            good_cols = list(df.iloc[source_breaks[i]])
            temp_dict = dict(zip(bad_cols, good_cols))
            d['{0}'.format(vals)].rename(columns=temp_dict, inplace=True)
            d['{0}'.format(vals)].reset_index(inplace=True, drop=True)
    
    # This for loop formats date-time columns to appropriate datatype.
    # This is by far the most time-instensive process of this function
    # Time savings could be realized if ONLY dates are needed and not time & date.
    #
    # I am also using this for loop to trim unneeded rows in the INSTAGRAM
    # and YOUTUBE dataframes.
    #
    # By using the lambda function and dictionary generator, I was able to achieve a
    # 4 second increase in processing speed.
    for key in d.keys():
        
        # Because INSTAGRAM and YOUTUBE have different column structures than all others,
        # set up an if/else structure to process these separately
        if key == 'instagram':
           
            # Dictionary generator to create a dates dictionary with the unique date string
            # from the original dataframe and associating the to_datetime formatted date to 
            # said string
            dates = {date:pd.to_datetime(date) for date in d[key]['Date'].unique()}
            
            # use a lambda function to apply the new dates to the dataframe
            d[key]['Date'] = d[key]['Date'].apply(lambda v: dates[v])
            
            # Now use an apply function to apply to_datetime to all entries in the 'Time' column
            d[key]['Time'] = d[key]['Time'].apply(pd.to_datetime, errors='ignore')
            
            #d[key][['Date', 'Time']] = d[key][['Date', 'Time']].apply(pd.to_datetime, errors='ignore')
            
            # Trim the dataframe to only relevant rows
            d[key] = d[key].iloc[:, 2:24]
        elif key == 'youtube':
            dates = {date:pd.to_datetime(date) for date in d[key]['Date'].unique()}
            d[key]['Date'] = d[key]['Date'].apply(lambda v: dates[v])
            d[key]['Time'] = d[key]['Time'].apply(pd.to_datetime, errors='ignore')
            #d[key][['Date', 'Time']] = d[key][['Date', 'Time']].apply(pd.to_datetime, errors='ignore')
            d[key] = d[key].iloc[:, 2:17]
        else:
            dates = {date:pd.to_datetime(date) for date in d[key]['Date(ET)'].unique()}
            d[key]['Date(ET)'] = d[key]['Date(ET)'].apply(lambda v: dates[v])
            d[key][['Time(ET)', 'LocalTime']] = d[key][['Time(ET)', 'LocalTime']].apply(pd.to_datetime, errors='ignore')
            d[key] = d[key].iloc[:, 2:]
            
    clean_cols(d)
    
    s_time = time.time()
    for key in d.keys():
        d['{0}'.format(key)]['Cleaned_Text'] = d['{0}'.format(key)]['Cleaned_Text'].apply(cleanup)
    e_time = time.time()
    print(e_time - s_time)
    
    files_out(d)
    
    return(d)

In [10]:
def files_out(d):
    # Create an output directory for the new dataframes
    # if the directory already exists, just pass
    try:
        os.makedirs('./data/clean_dfs/')
    except FileExistsError:
        pass
    
    # Set up a for loop to iterate through all dataframes in the dictionary
    # If an output file does NOT already exist for that dataframe, 
    # create a new output file
    #
    # If the output file DOES exist, open that file in 'append' mode
    # and append the new data to the end without header information.
    #
    # Keeping the pandas index in place, so when data is read in for manipulation later,
    # set index_col = 0
    #
    # Also, index will have to be reset upon initial read in later as this append method
    # does not reset the index.  I am doing this in the interest of time savings.
    
    for key in d.keys():
        if not path.exists('./data/clean_dfs/{0}.csv'.format(key)):
            print('file {0}.csv does not exist. Creating new file'.format(key))
            d[key].to_csv('./data/clean_dfs/{0}.csv'.format(key), index=False)
        else:
            print('file {0}.csv does exist. Appending new data to existing file'.format(key))
            df_temp = pd.read_csv('./data/clean_dfs/{0}.csv'.format(key))
            df_temp = df_temp.append(d[key], ignore_index=True)
            df_temp.to_csv('./data/clean_dfs/{0}.csv'.format(key), index=False)
    

In [11]:
def clean_cols(df):
    for key in df.keys():
        print(key)
        display(df['{0}'.format(key)].head())
    
        if key == 'instagram':
            df['{0}'.format(key)]['Cleaned_Text'] = df['{0}'.format(key)]['Contents']
    
        else:
            df['{0}'.format(key)]['Cleaned_Text'] = df['{0}'.format(key)]['Contents']
            if df['{0}'.format(key)]['Summary'] is not np.NaN:
                df['{0}'.format(key)]['Cleaned_Text'].fillna(df['{0}'.format(key)]['Summary'], inplace=True)
            if df['{0}'.format(key)]['Title'] is not np.NaN:
                df['{0}'.format(key)]['Cleaned_Text'].fillna(df['{0}'.format(key)]['Title'], inplace=True)
            if df['{0}'.format(key)]['Snippet'] is not np.NaN:
                df['{0}'.format(key)]['Cleaned_Text'].fillna(df['{0}'.format(key)]['Snippet'], inplace=True)
    return(df)

In [12]:
def cleanup(text):
    
    #rusw_df = pd.read_csv('./Jim/rusw.csv')
    #stopwords = list(rusw_df.columns[0:1001])
    #stopwords_string = ''
    #for
        #stopwords_string = stopwords_string + i +' '
        
    ru_stopwords = stopwords.words('russian')
    rustopwords = ' '.join(ru_stopwords[:])
    
    
    
    #cstart = time.time()
    # insert a space between hashtags
    
    text = re.sub("\#"," ", text)
    
    # remove urls 
    
    text = re.sub("(http|https)\:\/\/.*\s?","",text)
    
    # remove HTML tags
    
    text = re.sub("\<.*\>","",text)
    
    # remove punctuation
    
    text = text.translate(str.maketrans('', '', punctuation))
    #cend = time.time()
    #print(text)
    #print("cleanup time:",cend - cstart)
   
    text = re.sub(rustopwords, '', text)
    
    
    
    tokens = [token for token in text.lower().split() if token not in rustopwords\
            and token != " "]
    
    text = " ".join(tokens)
    
    
    #return text.strip()
    return text

In [13]:
start = time.time()
df = read_data()
end = time.time()
print(end - start)

C:\Users\595872\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\595872\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


news


C:\Users\595872\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Host,Link,Date(ET),Time(ET),LocalTime,Category,Author ID,Author Name,Author URL,Authority,Followers,Following,Age,Gender,Language,Country,Province/State,City,Location,Sentiment,Themes,Classifications,Entities,Alexa Rank,Alexa Reach,Title,Snippet,Contents,Summary,Bio,Unique ID,Post Source
0,http://sakhalinmedia.ru/,https://sakhalinmedia.ru/news/824534/,2019-06-20,2019-07-16 23:15:17,2019-06-20 23:15:00,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,ru,ru,NaN,NaN,ru,NEUTRAL,NaN,NaN,NaN,42565,42119,"Афиша выходного дня: Сабантуй в лучах ""Утра Ро...","ИА SakhalinMedia рассказывает, куда пойти и чт...","ИА SakhalinMedia рассказывает, куда пойти и чт...","ИА SakhalinMedia рассказывает, куда пойти и чт...",NaN,NaN,NaN
1,http://mirtesen.sputnik.ru/,http://mirtesen.sputnik.ru/blog/43460930378/Af...,2019-06-20,2019-07-16 23:11:37,2019-06-20 23:11:00,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,ru,ru,NaN,NaN,ru,NEUTRAL,NaN,NaN,NaN,57760,29160,"Афиша выходного дня: Сабантуй в лучах ""Утра Ро...","Спутник 21 июня, 5:40 Прекрасная погода и уйма...","Спутник 21 июня, 5:40 Прекрасная погода и уйма...","Спутник 21 июня, 5:40 Прекрасная погода и уйма...",NaN,NaN,NaN
2,https://news.myseldon.com/ru/,https://news.myseldon.com/ru/news/index/212361...,2019-06-20,2019-07-16 20:13:27,2019-06-20 20:13:00,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,ru,ru,NaN,NaN,ru,NEUTRAL,NaN,NaN,NaN,79537,18000,"Афиша выходного дня: Сабантуй в лучах ""Утра Ро...","ИА SakhalinMedia рассказывает, куда пойти и чт...","ИА SakhalinMedia рассказывает, куда пойти и чт...","ИА SakhalinMedia рассказывает, куда пойти и чт...",NaN,NaN,NaN
3,http://www.trud.ru/,http://www.trud.ru/article/21-06-2019/1377277_...,2019-06-20,2019-07-16 18:25:43,2019-06-20 18:25:00,NaN,NaN,Анна Владимирова все статьи автора,NaN,2,NaN,NaN,NaN,NaN,ru,ru,NaN,NaN,ru,NEUTRAL,business;finance;,Org.business;,Citigroup;,61151,28800,Ольга Кабо: Бессмысленно воспринимать меня сек...,"«Сама-то я человек XXI века, мне комфортно жит...","«Сама-то я человек XXI века, мне комфортно жит...","«Сама-то я человек XXI века, мне комфортно жит...",NaN,NaN,NaN
4,https://www.bragazeta.ru/,https://www.bragazeta.ru/world/2847395/,2019-06-20,2019-07-16 16:00:28,2019-06-20 16:00:00,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,ru,ru,NaN,NaN,ru,NEUTRAL,NaN,NaN,NaN,32305,59220,От «психотерапии» - к агрессивному НЛП-убежден...,"Естественно, разговор с населением неизбежно д...","Естественно, разговор с населением неизбежно д...","Естественно, разговор с населением неизбежно д...",NaN,NaN,NaN


forums


...

twitter


,Host,Link,Date(ET),Time(ET),LocalTime,Category,Author ID,Author Name,Author URL,Authority,Followers,Following,Age,Gender,Language,Country,Province/State,City,Location,Sentiment,Themes,Classifications,Entities,Alexa Rank,Alexa Reach,Title,Snippet,Contents,Summary,Bio,Unique ID,Post Source
0,http://www.twitter.com,http://twitter.com/diamondcircus/statuses/1141...,2019-06-20,2019-07-16 23:04:15,2019-06-20 23:04:00,NaN,diamondcircus,Бриллиантовый цирк,http://twitter.com/diamondcircus,4,441,2764,NaN,NaN,Russian,NaN,NaN,NaN,"Якутск, Саха Якутия республика",NEUTRAL,NaN,Product.grocery;,5 год;,NaN,NaN,NaN,Всемирный известный Итальянский цирк приглашае...,Всемирный известный Итальянский цирк приглашае...,Всемирный известный Итальянский цирк приглашае...,Государственный цирк - 25 лет!,1.1419E+18,Twitter for iPhone
1,http://www.twitter.com,http://twitter.com/feronianya/statuses/1141728...,2019-06-20,2019-07-16 11:25:14,2019-06-20 11:25:00,NaN,feronianya,любимая,http://twitter.com/feronianya,5,451,350,NaN,MALE,Russian,NaN,NaN,NaN,г. Нижний Тагил,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,"Самое интересное , что она так и не поняла сут...","Самое интересное , что она так и не поняла сут...","Самое интересное , что она так и не поняла сут...",д н о,1.14173E+18,Twitter for iPhone
2,http://www.twitter.com,http://twitter.com/IPIi38622976/statuses/11417...,2019-06-20,2019-07-16 09:55:32,2019-06-20 09:55:00,NaN,IPIi38622976,🆘I PI🆘 .i.,http://twitter.com/ipii38622976,4,128,297,NaN,NaN,Russian,NaN,NaN,NaN,NaN,NEUTRAL,NaN,Org.broadcaster;,RT;,NaN,NaN,NaN,"RT @vkovall: @zabanen_snova Ну. Трещит у неё, ...","RT @vkovall: @zabanen_snova Ну. Трещит у неё, ...","У неё в глазах тупизна, ну пускай се кукарекае...",NaN,1.14171E+18,Twitter for Android
3,http://www.twitter.com,http://twitter.com/SaviorMars/statuses/1141703...,2019-06-20,2019-07-16 09:43:38,2019-06-20 09:43:00,NaN,SaviorMars,Savior,http://twitter.com/saviormars,6,1152,186,NaN,NaN,Russian,NaN,NaN,NaN,"Москва, Россия",NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,как же это уже все заебало — сперва четырехчас...,как же это уже все заебало — сперва четырехчас...,NaN,NaN,1.1417E+18,Twitter for iPhone
4,http://www.twitter.com,http://twitter.com/vkovall/statuses/1141701650...,2019-06-20,2019-07-16 09:37:40,2019-06-20 09:37:00,NaN,vkovall,Vasiliy Koval,http://twitter.com/vkovall,6,1127,209,NaN,MALE,Russian,Russia,khanty-mansiy,surgut,"msk, spb, surgut, Russia.",NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,"@zabanen_snova Ну. Трещит у неё, но она же в п...","@zabanen_snova Ну. Трещит у неё, но она же в п...","У неё в глазах тупизна, ну пускай се кукарекае...",Если тебе плохо - вспомни Ленинград 1941-1942 ...,1.1417E+18,Twitter for iPhone


instagram


,Link,Date,Time,Author ID,Author Name,Language,Location,Contents,HashTags,Likes,Comments,Attachments,Brand Images,Object Images,Food Images,Scene Images,Selfie,Sentiment,Themes,Classifications,Entities,Unique ID
0,https://www.instagram.com/anya_evtyshenko/p/Br...,2018-12-10,2019-07-16 16:33:36,anya_evtyshenko,NaN,Russian,NaN,"Кто знает,что это???😁😁😅 Будем готгвиться！ А чт...",чер;cheer;cheerleading;воркаут;акробатика;work...,0,0,https://scontent.cdninstagram.com/vp/17fa926b8...,NaN,NaN,NaN,NaN,FALSE,NEUTRAL,sports;,Event.sport;,Воркаут;,201812IG18125542457976932289279
1,https://www.instagram.com/p/BrONHuSh0ap/,2018-12-10,2019-07-16 16:30:13,kostolesa,NaN,Russian,NaN,Поход в 🎪❤ #сын#сыночек#дети#детицветыжизни #х...,хорошеенастроение;follow;like;детицветыжизни;s...,0,0,https://scontent.cdninstagram.com/vp/996ae5daf...,NaN,NaN,NaN,NaN,FALSE,NEUTRAL,NaN,NaN,NaN,201812IG18123263095387943056683
2,https://www.instagram.com/p/BrOK94_g5wt/,2018-12-10,2019-07-16 16:11:23,ola_bond_,NaN,Russian,NaN,"Расскажу по-быстрому, так как уже, как ""варенн...",stradivarius;zara;леруамерлен;pullbeer,0,0,https://scontent.cdninstagram.com/vp/edcbdf9e3...,NaN,NaN,NaN,NaN,FALSE,NEUTRAL,business;finance;,Org.business;,Zara;,201812IG18124045939827507806156
3,https://www.instagram.com/p/BrOCHzYnowL/,2018-12-10,2019-07-16 14:54:06,show_duet_delight,NaN,Russian,NaN,Ходулист на ваше мероприятие 🤸‍♂️🤹🏼‍♀️#шоупрог...,шоуднепр;шоузапорожье;шоупрограмма;артисты;цир...,0,0,https://scontent.cdninstagram.com/vp/5d441f66e...,NaN,NaN,NaN,NaN,FALSE,NEUTRAL,NaN,NaN,NaN,201812IG181263125133205861379
4,https://www.instagram.com/p/BrOA0TGlpDi/,2018-12-10,2019-07-16 14:42:42,katya__vatsyk,NaN,Russian,NaN,"Намечается очень активная неделя, упорно труди...",харьков;комбинезон;украина;комбинезоны;винница...,0,0,https://scontent.cdninstagram.com/vp/b81db9961...,NaN,NaN,NaN,NaN,FALSE,NEUTRAL,NaN,NaN,NaN,201812IG18128884113811840777466


48.46433091163635
file news.csv does not exist. Creating new file
file forums.csv does not exist. Creating new file
file tumblr.csv does not exist. Creating new file
file twitter.csv does not exist. Creating new file
file instagram.csv does not exist. Creating new file
66.39062881469727


In [ ]:
rusw_df = pd.read_csv('./Jim/rusw.csv')
stopwords = list(rusw_df.columns[0:1001])
stopwords_string = ''
for i in stopwords:
    stopwords_string = stopwords_string + i +','
print(stopwords_string)


In [14]:
df['twitter'].head()

,Host,Link,Date(ET),Time(ET),LocalTime,Category,Author ID,Author Name,Author URL,Authority,Followers,Following,Age,Gender,Language,Country,Province/State,City,Location,Sentiment,Themes,Classifications,Entities,Alexa Rank,Alexa Reach,Title,Snippet,Contents,Summary,Bio,Unique ID,Post Source,Cleaned_Text
0,http://www.twitter.com,http://twitter.com/diamondcircus/statuses/1141...,2019-06-20,2019-07-16 23:04:15,2019-06-20 23:04:00,NaN,diamondcircus,Бриллиантовый цирк,http://twitter.com/diamondcircus,4,441,2764,NaN,NaN,Russian,NaN,NaN,NaN,"Якутск, Саха Якутия республика",NEUTRAL,NaN,Product.grocery;,5 год;,NaN,NaN,NaN,Всемирный известный Итальянский цирк приглашае...,Всемирный известный Итальянский цирк приглашае...,Всемирный известный Итальянский цирк приглашае...,Государственный цирк - 25 лет!,1.1419E+18,Twitter for iPhone,всемирный известный итальянский цирк приглашае...
1,http://www.twitter.com,http://twitter.com/feronianya/statuses/1141728...,2019-06-20,2019-07-16 11:25:14,2019-06-20 11:25:00,NaN,feronianya,любимая,http://twitter.com/feronianya,5,451,350,NaN,MALE,Russian,NaN,NaN,NaN,г. Нижний Тагил,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,"Самое интересное , что она так и не поняла сут...","Самое интересное , что она так и не поняла сут...","Самое интересное , что она так и не поняла сут...",д н о,1.14173E+18,Twitter for iPhone,самое интересное поняла сути главный аргумент ...
2,http://www.twitter.com,http://twitter.com/IPIi38622976/statuses/11417...,2019-06-20,2019-07-16 09:55:32,2019-06-20 09:55:00,NaN,IPIi38622976,🆘I PI🆘 .i.,http://twitter.com/ipii38622976,4,128,297,NaN,NaN,Russian,NaN,NaN,NaN,NaN,NEUTRAL,NaN,Org.broadcaster;,RT;,NaN,NaN,NaN,"RT @vkovall: @zabanen_snova Ну. Трещит у неё, ...","RT @vkovall: @zabanen_snova Ну. Трещит у неё, ...","У неё в глазах тупизна, ну пускай се кукарекае...",NaN,1.14171E+18,Twitter for Android,rt vkovall zabanensnova трещит неё принципе ту...
3,http://www.twitter.com,http://twitter.com/SaviorMars/statuses/1141703...,2019-06-20,2019-07-16 09:43:38,2019-06-20 09:43:00,NaN,SaviorMars,Savior,http://twitter.com/saviormars,6,1152,186,NaN,NaN,Russian,NaN,NaN,NaN,"Москва, Россия",NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,как же это уже все заебало — сперва четырехчас...,как же это уже все заебало — сперва четырехчас...,NaN,NaN,1.1417E+18,Twitter for iPhone,заебало — сперва четырехчасовой цирк маразмати...
4,http://www.twitter.com,http://twitter.com/vkovall/statuses/1141701650...,2019-06-20,2019-07-16 09:37:40,2019-06-20 09:37:00,NaN,vkovall,Vasiliy Koval,http://twitter.com/vkovall,6,1127,209,NaN,MALE,Russian,Russia,khanty-mansiy,surgut,"msk, spb, surgut, Russia.",NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,"@zabanen_snova Ну. Трещит у неё, но она же в п...","@zabanen_snova Ну. Трещит у неё, но она же в п...","У неё в глазах тупизна, ну пускай се кукарекае...",Если тебе плохо - вспомни Ленинград 1941-1942 ...,1.1417E+18,Twitter for iPhone,zabanensnova трещит неё принципе тупая неё гла...
